In [1]:
# The original matrix search was written by H Nakano 20220908.
# The search algorithm levaraging base20 indexing was written by Soki Nakano.
# Performance improvements, code cleanup and flexible use of any peptide size written by Maurizio Camagna 20230413

import pandas as pd
import numpy as np
import sys, csv, re

from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

## Changes made from previous version
- The length of the the peptide is now inferred from the number of columns in the *score_table* file
- Improved preformance: The previous version calculated all scores for all possible peptides of a given size and stored the scores in a large list. This was inefficent, since scores are calculated for peptides, that don't even occur in the proteom, and with larger peptides, this approach gets exponentially more inefficient. Instead, the scores are now only calculated for the peptides found in the fasta file. Once a score was calculated, it is stored in the list to skip the score calculation the next time the same peptide is encountered
- The file *score_ref_name* fulfilled no purpose and was removed (it was only written, but never read)
- Unused lines of code were removed
- The position is correctly reported in the output file
- Added an additional, compact output file
    - One row per protein
    - Contains total peptide score of each protein
    - Peptide matches are written as upper-case in the protein sequence
- Added some info on the scoring table (min/max scores)

## Notes:
- The column names of the score table are arbitrary, except for the first column, which must be named **AA**
- The order of the rows in the score table is also arbitrary, as the script will sort the file alphabetically

# Inputs

In [2]:
#score_table_file = 'Input_file_6mer.csv' #name of score table file.
score_table_file = 'Input_file.csv' #name of score table file.
fasta_in = 'Human_proteins.fasta' # name of  multifasta file for search.
minimum_score = 12 # specify minimum score 
csv_name = 'output.csv' # output file name of Q and its surrounding without Q close to the C-terminal

<br><br><br><br>

In [3]:
score_table = pd.read_csv(score_table_file)
score_table

,AA,-1,Q,1,2,3
0,D,0.627703,0,0.779207,0.795582,0.685065
1,E,0.684350,0,0.598563,0.842955,0.543147
2,N,0.741804,0,0.701087,0.843187,0.796826
3,Q,1.497140,10,1.035081,1.012754,0.704695
4,P,0.918332,0,0.567865,0.603739,0.676449
5,Y,0.964919,0,1.106978,1.679671,0.871349
6,W,1.093472,0,0.983508,1.073539,0.775425
7,S,0.916496,0,1.498919,0.796677,0.792019
8,T,0.823655,0,0.853654,0.929890,0.870773
9,G,0.990309,0,0.711916,0.950565,0.781286


Let's take a look at the scoring matrix and confirm that it is correct, as well as help us choose a minimum_score

In [4]:
best_peptide = ""
worst_peptide = ""

for col in score_table.columns[1:]:
    best_peptide += score_table.iloc[np.argmax(score_table[col]), 0]
    worst_peptide += score_table.iloc[np.argmin(score_table[col]), 0]

    
max_score = score_table.iloc[:,1:].max().sum()
min_score = score_table.iloc[:,1:].min().sum()

print("According to the score table, the min/max possible scores are:")
print(f"Max: {max_score}\t{best_peptide}")
print(f"Min: {min_score}\t{worst_peptide}")

According to the score table, the min/max possible scores are:
Max: 16.568911359	HQSYV
Min: 1.9881229449999998	KDKFK


In [5]:
score_table_list = score_table.sort_values("AA").values.tolist()
n = len(score_table.columns)-1 #n is the number of amino acids in the peptide

In [6]:
pep_num_list = [("A", 0), ("C", 1), ("D", 2), ("E", 3), ("F", 4), ("G", 5), ("H", 6), \
          ("I", 7), ("K", 8), ("L", 9), ("M", 10), ("N", 11), ("P", 12), ("Q", 13),\
          ("R", 14), ("S", 15), ("T", 16), ("V", 17), ("W", 18), ("Y", 19)]
pep_num_dict=dict(pep_num_list) 

In [7]:
# output pep is removed at 2nd Q
def num2pep(num, n=6):
    """Generates peptides of length n from numbers. Default: 6 
    0 -> AAAAAA, 1 -> AAAAAC ... (20**6)-1 -> YYYYYY
    """
    
    base_20 = np.base_repr(num, 20)   
    out = ""
    for char in str(base_20).zfill(n):
        base_10 = int(char, 20)        
        out += pep_num_list[base_10][0] 
    return out

In [8]:
def pep2num(pep, n=6):
    """Converts a peptide sequence to a unique number, which allows a lookup in an array"""
    num = 0
    for i in range(n):
        if(pep[i] not in pep_num_dict):
            raise Exception("{} is not in pep_num_dict".format(pep[i]))
        num += pep_num_dict[pep[i]] * 20 **((n-1)-i) #10進法に変換 
    return num

In [9]:
def pep2score(pep):
    """Calculates the score for a given peptide."""
    score = 0
    for i, item in enumerate(pep):
        num = pep_num_dict[item]
        score += score_table_list[num][i+1]
    return score

In [10]:
#let's generate an empty array which will hold our peptide scores. For now, all scores are set to -1.
#if we encouter a peptide for the first time in a protein, we'll calculate the score and store it in this list
#to re-use it later. The position of a peptide in the list is calculated via pep2num()
score_ref = [-1]*(20**n)

In [11]:
out = []

for record in SeqIO.parse(fasta_in, 'fasta'):
    id_part = record.id
    desc_part = record.description
    seq = str(record.seq)
    
    for i in range (len(seq)-n+1):
        pep = seq[i:i+n]
        
        try:    
            num = pep2num(pep, n)
            score = score_ref[num]
            if score == -1:
                #the score has not been calculated yet
                score = pep2score(pep)
                score_ref[num] = score #let's store the score, to allow a quick lookup if we encouter the peptide again
            
        except Exception as e:
            score = 0
        if score > minimum_score:

            seq_t = [id_part, desc_part, i+1, score, pep, seq]
            out.append(seq_t) 


Let's write the peptide result to a csv file

In [12]:
if len(out)>0:
    out = pd.DataFrame(out)
    out.columns = ["id","desc","posi","score","target","sequence"]
    out = out.set_index('id')
    out = out.sort_values(by='score', ascending=False)
    out.to_csv(csv_name)
else:
    print("No output file written: 0 hits")

<br>Let's also write a more compact output file (one row per protein)

In [13]:
if len(out)>0:
    #re-assure that the output is sorted by peptide scores
    out = out.sort_values(by='score', ascending=False)

    out_list = []
    out_list_columns = ["id", 'total score',"highest individual peptide score", 'peptide matches', 'unique peptide matches', 'sequence', 'peptides (peptide,score,count)']

    #group by each protein
    for protein_name, df in out.groupby(out.index):

        protein_score = df['score'].sum() #total peptide score for this protein
        peptide_matches = df['target'].count() #how many peptides matched
        unique_peptide_matches = df['target'].drop_duplicates().count() #how many peptides matches (no duplicates)

        highest_scoring_peptide = df['target'].iloc[0]
        highest_peptide_score = df['score'].iloc[0]

        #Let's write the protein sequence as lower case, 
        #except for positions where a peptide matches
        seq = df['sequence'][0].lower()
        for target in df['target']:
            seq = re.sub(target, target, seq, flags=re.IGNORECASE)


        peptide_dict = {}
        peptide_counts = df.groupby('target').count().iloc[:,0].to_dict()

        for peptide, score in zip(df['target'], df['score']):
            peptide_dict[peptide] = {"score":score, 'count':peptide_counts[peptide]}

        peptide_string = ""
        for peptide, value in peptide_dict.items():
            peptide_string+= f"{peptide},{round(value['score'],3)},{value['count']}|"
        peptide_string = peptide_string[:-1]


        out_list.append([protein_name, protein_score, highest_peptide_score, peptide_matches, unique_peptide_matches, seq, peptide_string])


    dense_out = pd.DataFrame(out_list)
    dense_out.columns = out_list_columns
    dense_out = dense_out.sort_values(by='total score', ascending=False)
    dense_out.to_csv(csv_name.replace(".csv", '.dense.csv'))